In [0]:
CREATE OR REFRESH STREAMING LIVE TABLE bronze_customers
AS SELECT * FROM cloud_files("/mnt/raw/customers", "csv", map("header", "true"));

In [0]:
CREATE OR REFRESH STREAMING LIVE TABLE bronze_employees
AS SELECT * FROM cloud_files("/mnt/raw/employees", "csv", map("header", "true"));

In [0]:
CREATE OR REFRESH STREAMING LIVE TABLE bronze_products
AS SELECT * FROM cloud_files("/mnt/raw/products", "csv", map("header", "true"));

In [0]:
CREATE OR REFRESH STREAMING TABLE silver_customers;
APPLY CHANGES INTO
  live.silver_customers
FROM
  stream(LIve.bronze_customers)
KEYS
  (customer_id)
SEQUENCE BY
  updated_at
COLUMNS * EXCEPT
  (_rescued_data
)
STORED AS
  SCD TYPE 1;

In [0]:
CREATE OR REFRESH STREAMING TABLE silver_employees;
APPLY CHANGES INTO
  live.silver_employees
FROM
  stream(LIve.bronze_employees)
KEYS
  (employee_id)
SEQUENCE BY
  updated_at
COLUMNS * EXCEPT
  (_rescued_data
)
STORED AS
  SCD TYPE 2;

In [0]:
create OR REFRESH streaming table silver_products
(
 constraint valid_category expect (category is not null) on violation drop row
)
as
select product_id
,name
,category
,price
,in_stock from Stream(LIVE.bronze_products)